# CityLearn with Garage
Name: Shota Takeshima
* garage is a toolkit for developing and evaluating reinforcement learning algorithms, and an accompanying library of state-of-the-art implementations built using that toolkit.
* First, I try to confirm SAC from garage can works with CityLearn.

## Installation: Garage
* garage supports python 3.6 or later.

In [1]:
!python --version

Python 3.6.9


In [2]:
# install garage
!echo "abcd" > mujoco_fake_key
!rm -rf garage
!git clone --depth 1 https://github.com/rlworkgroup/garage/
# in this execution of batch script, an error occurs...
!cd garage && bash scripts/setup_colab.sh --mjkey ../mujoco_fake_key --no-modify-bashrc > /dev/null!

Cloning into 'garage'...
remote: Enumerating objects: 755, done.
remote: Counting objects: 100% (755/755), done.
remote: Compressing objects: 100% (663/663), done.
remote: Total 755 (delta 203), reused 220 (delta 78), pack-reused 0
Receiving objects: 100% (755/755), 2.97 MiB | 30.39 MiB/s, done.
Resolving deltas: 100% (203/203), done.
start of setup_colab.sh




debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 88.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 


Cloning into '/tmp/tmp.3tpDeW1Re5/glfw'...
remote: Enumerating objects: 61, done.
remote: Counting objects: 100% (61/61), done.
remote: Compressing objects: 100% (26/26), done.
remote

In [4]:
!garage examples

2021-02-05 17:05:59.375722: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
# tf
tf/ddpg_pendulum.py (garage.examples.tf.ddpg_pendulum)
tf/dqn_cartpole.py (garage.examples.tf.dqn_cartpole)
tf/dqn_pong.py (garage.examples.tf.dqn_pong)
tf/erwr_cartpole.py (garage.examples.tf.erwr_cartpole)
tf/her_ddpg_fetchreach.py (garage.examples.tf.her_ddpg_fetchreach)
tf/multi_env_ppo.py (garage.examples.tf.multi_env_ppo)
tf/multi_env_trpo.py (garage.examples.tf.multi_env_trpo)
tf/ppo_memorize_digits.py (garage.examples.tf.ppo_memorize_digits)
tf/ppo_pendulum.py (garage.examples.tf.ppo_pendulum)
tf/reps_gym_cartpole.py (garage.examples.tf.reps_gym_cartpole)
tf/resume_training.py (garage.examples.tf.resume_training)
tf/rl2_ppo_halfcheetah.py (garage.examples.tf.rl2_ppo_halfcheetah)
tf/rl2_ppo_halfcheetah_meta_test.py (garage.examples.tf.rl2_ppo_halfcheetah_meta_test)
tf/rl2_ppo_metaworld_ml10.py (garage.examples.tf.rl2_ppo_metaworld

**Restart this notebook here.** it's needed to recognize the installed packages.

## Installation: CityLearn_garage

To use CityLearn in garage framework, we need to modify CityLearn environment accoding to the [garage document](https://garage.readthedocs.io/en/latest/user/implement_env.html).

* I modified the CityLearn env as CityLearn_garage and push it to my private git repository.

In [5]:
!rm -rf ./CityLearn_garage/
!git clone https://github.com/shttksm/CityLearn_garage.git

Cloning into 'CityLearn_garage'...
remote: Enumerating objects: 87, done.
remote: Counting objects: 100% (87/87), done.
remote: Compressing objects: 100% (80/80), done.
remote: Total 87 (delta 6), reused 84 (delta 6), pack-reused 0
Unpacking objects: 100% (87/87), done.


In [1]:
import sys
sys.path.append("./CityLearn_garage")

from citylearn import CityLearn
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np

!pip install stable_baselines3

import os
import gym
import numpy as np
from stable_baselines3 import SAC
from stable_baselines3.sac.policies import MlpPolicy as MlpPolicy_SAC
from stable_baselines3.common.callbacks import BaseCallback
import matplotlib.pyplot as plt
from pathlib import Path
import time
import pandas as pd
import seaborn as sns

In [2]:
# Select the climate zone and load environment
climate_zone = 1
data_path = Path("./CityLearn_garage/data/Climate_Zone_"+str(climate_zone))
building_attributes = data_path / 'building_attributes.json'
weather_file = data_path / 'weather_data.csv'
solar_profile = data_path / 'solar_generation_1kW.csv'
building_state_actions = './CityLearn_garage/buildings_state_action_space.json'
building_ids = ["Building_1","Building_2","Building_3","Building_4","Building_5","Building_6","Building_7","Building_8","Building_9"]
objective_function = ['ramping','1-load_factor','average_daily_peak','peak_demand','net_electricity_consumption']
env = CityLearn(data_path, 
                building_attributes, 
                weather_file, 
                solar_profile, 
                building_ids, 
                buildings_states_actions = building_state_actions, 
                cost_function = objective_function,
                central_agent = True,
                verbose=1)

observations_spaces, actions_spaces = env.get_state_action_spaces()
building_info = env.get_building_information() 

See buildings_state_action_space.json. Basically, there are 28 variables that composes the state. For example, Building_1 uses 26 variables out of 28 possible variables as its state. On the other hand, Building_3 has 25 elements that is one less than Building_1's. This is because Building_3 has only cooling_storage, not have dhw_storage. So, dhw_storage_soc is not a variable that composes Building_3's state.

In [3]:
observations_spaces

[Box(26,),
 Box(26,),
 Box(25,),
 Box(26,),
 Box(26,),
 Box(26,),
 Box(26,),
 Box(26,),
 Box(26,)]

In [4]:
actions_spaces

[Box(2,),
 Box(2,),
 Box(1,),
 Box(1,),
 Box(2,),
 Box(2,),
 Box(2,),
 Box(2,),
 Box(2,)]

In [5]:
#building_info

Above state and action spaces are for multi agents RI, below ones are for the single central agent. The number of variables included in the action space is the sum of `actions_spaces` for muliple agent. The number of variables included in the state space is not corresponding to the sum of `observations_spaces` because all building has the same variable like `t_out`, outdoor temperature. So, one of them is includes as a state variable, others are omitted.

From `citylearn.py`
```python
# If the agent is centralized, we append the states avoiding repetition. I.e. if multiple buildings share the outdoor temperature as a state, we only append i\t once to the states of the central agent. The variable s_appended is used for this purpose. 
```

In [6]:
env.action_space

Box(16,)

In [7]:
env.observation_space

Box(81,)

## Modifications for garage
This is memos to remember what I modified to the original source code.

* The modification instruction is [here](https://garage.readthedocs.io/en/latest/user/implement_env.html).
* However, CityLearn is already a subclass of gym.Env. First, I just use a wrapper class [GymEnv](https://github.com/rlworkgroup/garage/blob/master/src/garage/envs/gym_env.py) in garage.

  ```python
  import gym
  ...
  class CityLearn(gym.Env):   
  ```

## Confirming if CityLearn works correctly
**If you modify CityLearn to make it suited to garage, check if the CityLearn works similarly as before it is modified.**
* confirm if my modifications are correct, using SAC from stable baseline3 packages
* This is a result from the original CityLearn code. (after fixing the reward_function.py. See Appendix.)
![image](https://user-images.githubusercontent.com/56372825/107001085-06de9c00-6757-11eb-8fe8-a2fa9bd642ce.png)



In [7]:
%%time
env.reset()
agent = SAC(MlpPolicy_SAC, env, verbose=0, learning_rate=0.001, gamma=0.99, tau=3e-4, batch_size=2048, learning_starts=8759)
agent.learn(total_timesteps=8760 * 12, log_interval=1000)

Cumulated reward: -113012696825.10913
Cumulated reward: -106994741264.02734
Cumulated reward: -109388770632.78522
Cumulated reward: -101347014538.57733
Cumulated reward: -92452868926.78209
Cumulated reward: -89046026243.33894
Cumulated reward: -88986360804.9545
Cumulated reward: -90421942685.15973
Cumulated reward: -94936514890.54431
Cumulated reward: -91580495960.89917
Cumulated reward: -93396848292.75578
Cumulated reward: -90913042180.85068
CPU times: user 24min 4s, sys: 49.9 s, total: 24min 54s
Wall time: 25min 2s


In [8]:
obs = env.reset()
dones = False
counter = []
while dones==False:
    action, _states = agent.predict(obs)
    obs, rewards, dones, info = env.step(action)
    counter.append(rewards)
env.cost()

Cumulated reward: -91229984822.5988


./CityLearn_garage/citylearn.py:527: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  self.state = np.array(self.state)
./CityLearn_garage/citylearn.py:41: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(a)
./CityLearn_garage/citylearn.py:449: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  self.state = np.array(self.state)


{'1-load_factor': 0.9533623752498285,
 'average_daily_peak': 0.92366856,
 'net_electricity_consumption': 1.0157526,
 'peak_demand': 1.1265297,
 'ramping': 0.859761,
 'total': 0.9758148371677329}

## CityLearn with garage

### SAC : Pending Todo, clarify each functions and their parameters
* referring to the [SAC example of garage](https://github.com/rlworkgroup/garage/blob/master/src/garage/examples/torch/sac_half_cheetah_batch.py)

In [8]:
env.reset()
pass

In [9]:
"""This is an example to train a task with SAC algorithm written in PyTorch."""
import numpy as np
import torch
from torch import nn
from torch.nn import functional as F

from garage import wrap_experiment
from garage.envs import GymEnv, normalize
from garage.experiment import deterministic
from garage.replay_buffer import PathBuffer
from garage.sampler import FragmentWorker, LocalSampler
from garage.torch import set_gpu_mode
from garage.torch.algos import SAC
from garage.torch.policies import TanhGaussianMLPPolicy, DeterministicMLPPolicy, GaussianMLPPolicy
from garage.torch.q_functions import ContinuousMLPQFunction
from garage.trainer import Trainer

In [10]:
import warnings
warnings.filterwarnings("ignore")

@wrap_experiment(snapshot_mode='none')
def sac_citylearn(ctxt=None, seed=1, env=None):
    """Set up environment and algorithm and run the task.
    Args:
        ctxt (garage.experiment.ExperimentContext): The experiment
            configuration used by Trainer to create the snapshotter.
        seed (int): Used to seed the random number generator to produce
            determinism.
    """
    deterministic.set_seed(seed)
    trainer = Trainer(snapshot_config=ctxt)
    # add the optional arg, max_episode_length
    env = GymEnv(env, max_episode_length=env.simulation_period[1])

    policy = TanhGaussianMLPPolicy(
        env_spec=env.spec,
        hidden_sizes=[256, 256],
        hidden_nonlinearity=nn.ReLU,
        output_nonlinearity=None,
        min_std=np.exp(-20.),
        max_std=np.exp(2.),
    )

    qf1 = ContinuousMLPQFunction(env_spec=env.spec,
                                 hidden_sizes=[256, 256],
                                 hidden_nonlinearity=F.relu)

    qf2 = ContinuousMLPQFunction(env_spec=env.spec,
                                 hidden_sizes=[256, 256],
                                 hidden_nonlinearity=F.relu)

    replay_buffer = PathBuffer(capacity_in_transitions=int(1e6))

    sampler = LocalSampler(agents=policy,
                           envs=env,
                           max_episode_length=env.spec.max_episode_length,
                           worker_class=FragmentWorker)

    sac = SAC(env_spec=env.spec,
              policy=policy,
              qf1=qf1,
              qf2=qf2,
              sampler=sampler,
              gradient_steps_per_itr=1000,
              max_episode_length_eval=1000,
              replay_buffer=replay_buffer,
              min_buffer_size=1e4,
              target_update_tau=5e-3,
              discount=0.99,
              buffer_batch_size=256,
              reward_scale=1.,
              steps_per_epoch=1)

    if torch.cuda.is_available():
        set_gpu_mode(True)
    else:
        set_gpu_mode(False)
    sac.to()
    trainer.setup(algo=sac, env=env)
    trainer.train(n_epochs=10, batch_size=1000)

sac_citylearn(seed=521, env=env)

2021-02-05 17:54:43 | [sac_citylearn] Logging to /content/data/local/experiment/sac_citylearn_5
2021-02-05 17:56:00 | [sac_citylearn] Obtaining samples...
2021-02-05 17:56:34 | [sac_citylearn] epoch #0 | Saving snapshot...
2021-02-05 17:56:34 | [sac_citylearn] epoch #0 | Saved
2021-02-05 17:56:34 | [sac_citylearn] epoch #0 | Time 34.00 s
2021-02-05 17:56:34 | [sac_citylearn] epoch #0 | EpochTime 34.00 s
----------------------------------  ---------------
AlphaTemperature/mean                   1.38377
Average/TrainAverageReturn             -3.27839e+06
Evaluation/AverageDiscountedReturn     -3.3824e+08
Evaluation/AverageReturn               -3.03338e+09
Evaluation/Iteration                    0
Evaluation/MaxReturn                   -3.03338e+09
Evaluation/MinReturn                   -3.03338e+09
Evaluation/NumEpisodes                 10
Evaluation/StdReturn                    0
Evaluation/TerminationRate              0
Policy/Loss                             5.8663e+06
QF/Qf1Loss     

Todo: After training correctly -> How to use the trained policy: https://garage.readthedocs.io/en/latest/user/reuse_garage_policy.html

### TRPO

In [18]:

import torch
from torch.nn import functional as F

from garage import wrap_experiment
from garage.envs import GymEnv
from garage.experiment.deterministic import set_seed
from garage.sampler import LocalSampler
from garage.torch.algos import TRPO
from garage.torch.policies import GaussianMLPPolicy
from garage.torch.value_functions import GaussianMLPValueFunction
from garage.trainer import Trainer


@wrap_experiment
def trpo_citylearn(ctxt=None, seed=1, env=env):
    """Train TRPO with InvertedDoublePendulum-v2 environment.
    Args:
        ctxt (garage.experiment.ExperimentContext): The experiment
            configuration used by Trainer to create the snapshotter.
        seed (int): Used to seed the random number generator to produce
            determinism.
    """
    set_seed(seed)
    env = GymEnv(env, max_episode_length=env.simulation_period[1])

    trainer = Trainer(ctxt)

    policy = GaussianMLPPolicy(env.spec,
                               hidden_sizes=[256, 256],
                               hidden_nonlinearity=F.relu,
                               output_nonlinearity=None)

    value_function = GaussianMLPValueFunction(env_spec=env.spec,
                                              hidden_sizes=(256, 256),
                                              hidden_nonlinearity=F.relu,
                                              output_nonlinearity=None)

    sampler = LocalSampler(agents=policy,
                           envs=env,
                           max_episode_length=env.spec.max_episode_length)

    algo = TRPO(env_spec=env.spec,
                policy=policy,
                value_function=value_function,
                sampler=sampler,
                discount=0.99,
                center_adv=False)

    trainer.setup(algo, env)
    trainer.train(n_epochs=10, batch_size=1000)


trpo_citylearn(seed=1)

2021-02-05 10:06:05 | [trpo_citylearn] [trpo_citylearn] [trpo_citylearn] [trpo_citylearn] [trpo_citylearn] Logging to /content/data/local/experiment/trpo_citylearn_6


/content/garage/src/garage/experiment/deterministic.py:37: UserWarning: Enabeling deterministic mode in PyTorch can have a performance impact when using GPU.
  'Enabeling deterministic mode in PyTorch can have a performance '


2021-02-05 10:06:22 | [trpo_citylearn] [trpo_citylearn] [trpo_citylearn] [trpo_citylearn] [trpo_citylearn] Obtaining samples...
Cumulated reward: -116099218698.46945
2021-02-05 10:06:36 | [trpo_citylearn] [trpo_citylearn] [trpo_citylearn] [trpo_citylearn] [trpo_citylearn] epoch #0 | Saving snapshot...
2021-02-05 10:06:42 | [trpo_citylearn] [trpo_citylearn] [trpo_citylearn] [trpo_citylearn] [trpo_citylearn] epoch #0 | Saved
2021-02-05 10:06:42 | [trpo_citylearn] [trpo_citylearn] [trpo_citylearn] [trpo_citylearn] [trpo_citylearn] epoch #0 | Time 20.34 s
2021-02-05 10:06:42 | [trpo_citylearn] [trpo_citylearn] [trpo_citylearn] [trpo_citylearn] [trpo_citylearn] epoch #0 | EpochTime 20.34 s
-----------------------------------  --------------
Evaluation/AverageDiscountedReturn     -3.94019e+08
Evaluation/AverageReturn               -1.16099e+11
Evaluation/Iteration                    0
Evaluation/MaxReturn                   -1.16099e+11
Evaluation/MinReturn                   -1.16099e+11
Eval

In [14]:
normalize

garage.envs.normalized_env.NormalizedEnv

## Appendix
### Reward for single central agentb:
* I observed the cost score became worse as the reward increased or the number of training epochs increased.
* In step function of CityLearn class: `building.current_net_electricity_demand` stores each building's demand. And, it is positive if its (solar) generation is greater than its consumption.

```python
# Adding loads from appliances and subtracting solar generation to the net electrical load of each building                                                       
building_electric_demand = round(_electric_demand_cooling + _electric_demand_dhw + _non_shiftable_load - _solar_generation, 4)                                    

# Electricity consumed by every building                                                                                                                          
building.current_net_electricity_demand = building_electric_demand                                                                                                
self.buildings_net_electricity_demand.append(-building_electric_demand) # >0 if solar generation > electricity consumption  
```

* In `reward_function.py`: let `buildings_net_electricity_demand` be [-10, -20]. This means there are two buildings and each buildings consumes more than its generation. So, this is a bad case.

1. reward_ = reward_ = -np.array(electricity_demand).sum(): $- (-10 - 20) = 30$ 
2. reward_ = max(0, reward_): $\max(0, 30) = 30$
3. reward_ = reward_**3: $30^3 = 27000$

??? On the other hand, let `buildings_net_electricity_demand` be [10, 20]. Clearly this is a good case!

1. reward_ = reward_ = -np.array(electricity_demand).sum(): $- (10 + 20) = -30$ 
2. reward_ = max(0, reward_): $\max(0, -30) = 0$
3. reward_ = reward_**3: $0^3 = 0$

まじで頭悪い．I'll modify this code.

```python
# Reward function for the single-agent (centralized) agent                                                                                                                        
def reward_function_sa(electricity_demand):                                                                                                                                       
                                                                                                                                                                                  
    reward_ = -np.array(electricity_demand).sum()                                                                                                                                 
    reward_ = max(0, reward_)                                                                                                                                                     
    reward_ = reward_**3.0                                                                                                                                                        
                                                                                                                                                                                  
    return reward_    
```
